In [17]:
from clickhouse_driver import Client
import pandas as pd
bqClient = Client(user='lkovacevic',password='b9Ea8XzdFr0w6Zo32i4Z',host='clickhouse.sofascore.ai',port='9000')
# Executing a query and fetching data
query = """with pom as (
select distinct awayteam_id as team_id from sports.event a
where toDate(a.startdate)>='2023-01-01' and toDate(a.startdate)<'2023-07-01'
and a.sport_id=1),
    pom2 as (
select distinct hometeam_id as team_id from sports.event a
where toDate(a.startdate)>='2023-01-01' and toDate(a.startdate)<'2023-07-01'
and a.sport_id=1
    )
select a.team_id as team_id from pom a
                 union distinct
       select b.team_id as team_id from pom2 b;"""
teams = bqClient.execute(query)
teams=pd.DataFrame(teams)
teams.columns=['team_id']
print(teams)


       team_id
0       110753
1       110747
2       110757
3        76561
4       110751
...        ...
23520   478687
23521   478688
23522   468346
23523   389663
23524   478700

[23525 rows x 1 columns]


In [28]:
query = """select user_account_id,teams,mcc from bq.mobileuser
where user_account_id is not null and
      toDate(created_at)<='2023-06-30' and toDate(updated_at)<='2023-06-30' and
      mcc in (216, 218, 219, 220, 221, 222, 226, 232, 262,276, 284, 293, 294, 297)
and empty(teams)==0 order by mcc;
"""
mobileusers = bqClient.execute(query)
mobileusers=pd.DataFrame(mobileusers)
mobileusers.columns=['user_account_id','teams','mcc']
print(mobileusers['teams'])
print(mobileusers.columns)

0                                                   [2672]
1        [2817, 1921, 5121, 4481, 2692, 1924, 1925, 192...
2        [6948, 4709, 5589, 6631, 3051, 231407, 3052, 2...
3        [2817, 1921, 4481, 2819, 2692, 1924, 1925, 192...
4                         [33, 2595, 2692, 2836, 72, 1916]
                               ...                        
52717                                   [2817, 3428, 1644]
52718    [7685, 35782, 2829, 6733, 17, 6355, 14486, 484...
52719                                   [4819, 6355, 6637]
52720                                             [451286]
52721                                         [4481, 4715]
Name: teams, Length: 52722, dtype: object
Index(['user_account_id', 'teams', 'mcc'], dtype='object')


In [29]:
def filter_array(arr, filter_set):
    return [x for x in arr if x in filter_set]
mobileusers['teams'] = mobileusers['teams'].apply(lambda arr: filter_array(arr, teams['team_id'].values))

In [34]:
mobileusers = mobileusers[mobileusers['teams'].apply(len) > 0]
print(mobileusers)

                user_account_id  ...  mcc
0      5baa6beac4cf3390d5f2634c  ...  216
1      5c570a522ea51fc2f0eeb074  ...  216
2      63ee62a8c5515eabcf8dc45d  ...  216
3      62c85f22f72e83f06deda337  ...  216
4      6118a5601ace54809414ef87  ...  216
...                         ...  ...  ...
52717  61a38d059b49b85d274efe64  ...  297
52718  63ea3d541eb830b623003d64  ...  297
52719  62606bb6c7ba4d5852bb1b34  ...  297
52720  5eefd9721132eb47be921c34  ...  297
52721  638769d9138aaf6561ee6e67  ...  297

[48686 rows x 3 columns]


In [36]:
def concatenate_arrays(arrays):
    result = []
    for array in arrays:
        result.extend(array)
    return result

# Group by 'id' and 'mcc' and apply the concatenation function
result = mobileusers.groupby(['user_account_id', 'mcc'])['teams'].apply(concatenate_arrays).reset_index()

print(result)

                user_account_id  ...                                              teams
0      5317146dada0894a148b457f  ...                 [2672, 2032, 17, 2697, 4715, 2829]
1      5317399483eba54b458b4568  ...                           [2687, 2885, 4715, 2032]
2      531743a5a56b7e50168b45a2  ...  [2705, 2729, 2714, 17, 282108, 2719, 2713, 279...
3      53179053ada089b22e8b45ad  ...                                       [4707, 2714]
4      53184b0eff4cf92a1d8b469b  ...  [2687, 42464, 2689, 2695, 2705, 2829, 2702, 27...
...                         ...  ...                                                ...
41970  649dc68d9e37ee1dd68f52a0  ...                                             [5152]
41971  649dfbc6eeeb5827bfce8262  ...                                            [55371]
41972  649ed174c3818ad8dd922921  ...                             [4713, 2697, 35, 1653]
41973  649ed7e4ac78e899d8c6405d  ...               [2036, 4715, 201688, 44, 2829, 3006]
41974  649f63f2f9fb7189af86d85d 

In [37]:
print(len(mobileusers['teams']))
print(len(mobileusers['user_account_id'].unique()))
print(len(result['teams']))
print(len(result['user_account_id'].unique()))

48686
41935
41975
41935


In [26]:
query = """
    select toMonday(toDate(startdate)) as week_start,groupArray(id) as event_id,groupArray(tuple(hometeam_id,awayteam_id)) as team_ids from sports.event where
        toDate(startdate)>='2023-06-01' and toDate(startdate)<='2023-06-30' and
        sport_id==1 group by  week_start;
"""
events = bqClient.execute(query)
events=pd.DataFrame(events)
events.columns=['week_start','event_id','team_ids']
print(events)

   week_start  ...                                           team_ids
0  2023-05-29  ...  [(251305, 191916), (191934, 168128), (191916, ...
1  2023-06-05  ...  [(191917, 191916), (251305, 191917), (191935, ...
2  2023-06-12  ...  [(251305, 284610), (191933, 168128), (191917, ...
3  2023-06-19  ...  [(465504, 289131), (465519, 465523), (290016, ...
4  2023-06-26  ...  [(420051, 386313), (420039, 420038), (465506, ...

[5 rows x 3 columns]


In [61]:
def find_matching_indices(array, tuple_array):
    matching_indices = [i for i, tup in enumerate(tuple_array) if any(item in tup for item in array)]
    return matching_indices

def retrieve_ids(indices):
    matched_ids = []
    for idx in indices:
        for i in range(len(week_events)):
            if i == idx:
                matched_ids.append(week_events[i])
    return matched_ids

first_week = events.loc[0, 'team_ids']
for i in range(1,5):
    week=events.loc[i-1, 'team_ids']
    week_events=events.loc[i-1, 'event_id']

    colname = f"recom_week{i}"
    result['matching_indices'] = result['teams'].apply(lambda array: find_matching_indices(array, first_week))
    result[colname] = result['matching_indices'].apply(retrieve_ids)


print(result)

[(251305, 191916), (191934, 168128), (191916, 251305), (191917, 191933), (284610, 191935), (213354, 85337), (135522, 462417), (389561, 199622), (462403, 188778), (462406, 462402), (287286, 383863), (420037, 465509), (465524, 288326), (462408, 251873), (420050, 318251), (420851, 420159), (420048, 420024), (465513, 289131), (389560, 462413), (420051, 385847), (420852, 465504), (465519, 465523), (462418, 256645), (420042, 420041), (420035, 420036), (420242, 420851), (317412, 465512), (88742, 135516), (465517, 465522), (420017, 465521), (465504, 465513), (251683, 465507), (320500, 465518), (420038, 420028), (420045, 386313), (420023, 420048), (282734, 135518), (420039, 420019), (290016, 420018), (420020, 465524), (252820, 462407), (465511, 287664), (288544, 420043), (420049, 420026), (384668, 287286), (462405, 243813), (288326, 420047), (465523, 465517), (6964, 4999), (465520, 420037), (465514, 465515), (420024, 420022), (420044, 420035), (389971, 32781), (420852, 317412), (318924, 465509)

In [62]:
print(result.columns)

Index(['user_account_id', 'mcc', 'teams', 'matching_indices', 'recom_1w',
       'recom_week1', 'recom_week2', 'recom_week3', 'recom_week4'],
      dtype='object')


In [56]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

X = pd.DataFrame(mobileusers['teams'].tolist())
X=X.fillna(0)
print(X)

k = 10 
knn = NearestNeighbors(n_neighbors=k)
knn.fit(X)
    week=events.loc[i-1, 'team_ids']

empty_indices = result[result['teams'].apply(len) == 0].index
for i in range(1,len(empty_indices)):
    colname = f"recom_week{i}"
    df['concatenated_array'] = df.apply(lambda row: row['array1'] + row['array2'] + row['array3'], axis=1)
    result.loc[empty_indices[i-1],colname]
query = X.iloc[empty_indices]  
distances, indices = knn.kneighbors(query)
nearest_neighbors = X.iloc[indices[0]]
print(nearest_neighbors)


          0       1       2       3       4    ...  394  395  396  397  398
0        2672     0.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0  0.0
1        2817  1921.0  5121.0  4481.0  2692.0  ...  0.0  0.0  0.0  0.0  0.0
2        4709  3051.0  3052.0  2829.0     0.0  ...  0.0  0.0  0.0  0.0  0.0
3        2817  1921.0  4481.0  2819.0  2692.0  ...  0.0  0.0  0.0  0.0  0.0
4          33  2595.0  2692.0  2836.0    72.0  ...  0.0  0.0  0.0  0.0  0.0
...       ...     ...     ...     ...     ...  ...  ...  ...  ...  ...  ...
48681    2817  1644.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0  0.0
48682    2829    17.0  6355.0  5149.0  5152.0  ...  0.0  0.0  0.0  0.0  0.0
48683    4819  6355.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0  0.0
48684  451286     0.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0  0.0
48685    4481  4715.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0  0.0

[48686 rows x 399 columns]
       0    1    2    3    4    5    6    ...  392  393  394